In [4]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다                  
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
#import chromedriver_autoinstaller as ca
import pandas as pd 
import time 

#Step 2. 사용자에게 검색 관련 정보들을 입력 받고 3. 파일 이름 정하기
query_txt ='빅데이터'
ft_name = "c:\\py_temp\\riss.txt"
fc_name = "c:\\py_temp\\riss.csv"
fx_name = "c:\\py_temp\\riss.xls"
collect_cnt = 50

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
driver = webdriver.Chrome()
driver.get('https://www.riss.kr/')

# time.sleep(5)
# driver.maximize_window()

#새 창(팝업창) 제어하기
main = driver.window_handles #열린 창의 개수 저장

#첫번째로 뜨는 창은 MAIN창
for handle in main:
    if handle != main[0] :
        driver.switch_to.window(handle) #저장한 개수만큼 창을 이동한다. (다음 창으로 이동한다?)
        time.sleep(1)
        driver.close()

driver.switch_to.window(main[0])

#Step 5. 자동으로 검색어 입력 후 6 학위논문 클릭
driver.find_element(By.ID,'query').send_keys(query_txt+ '\n')
driver.find_element(By.LINK_TEXT,'학위논문').click()
time.sleep(2)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
soup_1 = BeautifulSoup(driver.page_source, 'html.parser') #속성값 가져오기
content_1 = soup_1.find('div','srchResultListW').find_all('li')

for i in content_1 :
    print(i.get_text().replace("\n","")) # 태그 제외 텍스트만 가져오기

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox pd').find('span','num').get_text()
print('키워드 %s (으)로  %s 건 검색' %(query_txt,total_cnt))
collect_page_cnt = math.ceil(collect_cnt / 10)


#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
no = 1

for a in range(1, collect_page_cnt + 1) :
    
    soup_2 = BeautifulSoup(driver.page_source, 'html.parser')
    content_2 = soup_2.find('div','srchResultListW').find_all('div', "cont ml60")
    
    for b in content_2 : 
        #1. 논문제목 있을 경우만 - 터미널에 출력, 리스트에 저장, 파일에 작성 (txt)
        try :
            title = b.find('p','title').get_text()
        except :
            continue
        else :
            f = open(ft_name, 'a' , encoding="UTF-8")
            
            print('1.번호:',no)
            no2.append(no)
            f.write('\n'+'1.번호:' + str(no))

            print('2.논문제목:',title)
            title2.append(title)
            f.write('\n' + '2.논문제목:' + title)
            
            writer = b.find('span','writer').get_text()
            print('3.저자:',writer)
            writer2.append(writer)
            f.write('\n' + '3.저자:' + writer)

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            f.write('\n' + '4.소속기관:' + org + '\n')
            
            f.close( )
            
            no += 1
            print("\n")
            
            if no > collect_cnt : # 수집한 자료가 50건이 넘으면 멈춘다. 
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

    a += 1 
    b = str(a)

    try :
        driver.find_element(By.LINK_TEXT ,'%s' %b).click() # 다음 2, 3 페이지 등등을 누른다. 
    except :
        driver.find_element(By.LINK_TEXT('다음 페이지로')).click() #다음에 클릭할 값이 없으면 '다음 페이지로'를 누른다. 
        
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
# 리스트에 있는 것들을 데이터프레임에 저장한다. 
df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8" , engine='openpyxl')

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

driver.close()

첫째, 4차 산업혁명 기술이 확산되는 경영환경 변화로 경쟁자 압박, 규정의 지원이 새로운 핵심요인으로 발견되었으며, 이는 시장경쟁자의 선제적 빅데이터 도입에 따른 압박감과 빅데이터 활용을 지원할 규정지원의 중요성이 커져감에 따라 핵심요인으로 인식된 것이라고 분석된다. 둘째, 개인관점에서 모든 요인이 핵심요인으로 나타난 것은 ICT기반의 업무환경 속에 능동적으로 적응하여 성과를 향상시켜야 하는 조직원의 변화된 인식이 반영된 것으로 해석되며, 성공적인 빅데이터 활용을 위하여 조직원이 중요한 고려요인임을 확인할 수 있었다. 셋째, 빅데이터 사용경험이 있는 집단의 리더십, 전략기획 수준이 높고 그 차이가 유의하게 나타남으로써, 빅데이터를 활용하는 것이 경영품질 수준을 높일 수 있다는 영향관계를 처음으로 확인하였다. 따라서 이러한 연구결과를 바탕으로 정부차원에서 다양한 중소기업 업종에서 활용할 수 있는 범용 빅데이터 시스템을 개발하여 지원한다면, 중소기업이 경영품질 향상을 통해 기업규모를 성장시켜 나가는데 도움이 될 수 있을 것이다. 넷째, 국가품질상 도전과정을 통해 경영품질이 기업 내에 내재화 되고, 이를 기반으로 지속가능경영을 추구해 나갈 수 있도록 정부의 적극적인 국가품질상 진흥정책의 확대가 필요하다. 이와 같이 본 연구는 기업의 빅데이터 활용과 경영품질 향상에 대한 이론적, 실용적 시사점을 제공하였다.s of challenging the national quality awards. Thus, this study provided the theoretical and practical implications for the Use of Big Data and Improvement of Management Quality. 최근 세계경제포럼(WEF)에서 ‘4차 산업혁명’이라는 용어가 언급된 이후, 정보통신기술(ICT)기반의 새로운 차세대 산업혁명으로 대표되면서 전 세계로 빠르게 확산되고 있다. 초 연결, 초 지능의 특징에 의해 데이터가 폭발적으로 증가되고, 이를 의미 있는

c:\Users\ahyeo\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
